## O intuito desse código é verificar a representatividade de um conjunto de autores de obras catalogadas em acervos digitais(Museus do projeto Tainacan) e confirmar se é possível  fazer um processo de vinculação automática dos dados com outro repositório de informações(Wikidata).

No repositório Museubr há um número considerável de urls informadas, ocorre que não necessariamente isso indica que as informações do acervo estejam disponíveis para consumo e consulta, nesse sentido, como fonte de pesquisa foi verificado o projeto Tainacan: 
 
O Projeto Tainacan é fruto de iniciativa que promove os fundamentos de uma política nacional para acervos digitais (arquivos, bibliotecas e museus), e constitui a mais nova etapa do Programa Acervo em Rede, do Instituto Brasileiro de Museus (Ibram).     
http://www.museus.gov.br/acessoainformacao/acoes-e-programas/projeto-tainacan/

Nesse sentido ele é um excelente ponto de partida, para escolha dos repositórios de fonte de autores de obras, isso porque como ele possui uma API é possível recuperar dados nos formatos CSV, JSON e HTML além de seguir um mesmo padrão, mesmo se tratando de repositórios diferentes

In [0]:
#instalação dos pacotes necessários
#wget para baixar os arquivos
#qwikidata para consultas SPARQL na Wikidata
pip install wget
pip install qwikidata

Deve ser carregado o arquivo lista-museus.csv com as informações dos museus a serem processados compativeis com o Tainacan com os seguintes campos:



*   nome-> Nome do museu Ex: Museu Histórico Nacional 
*   museu_base_url -> nome inicial da url Ex: mhn
*   url -> url do acervo Ex: mhn.acervos.museus.gov.br
*   colecao -> código da coleção no wordpress Ex: 24
*   colecao -> formato fixo csv Ex: csv
*   qde_arquivos -> quantidade de listas disponíveis no wordpress Ex: 6
*   token_seguranca -> token do wordpress(deve ser sempre renvado) Ex: 5e7b4c00ce
*   qde_itens -> Quantidade de intes informado no acervo Ex: 502












	



In [7]:
#CODIGO DE IMPORTAÇÃO DOS ARQUIVOS DO MUSEU CONFORME ARQUIVO lista-museus.csv 
import wget
import os.path
import numpy
import pandas as pd

lista_info_leitura = []

def baixar_arquivo(file_path,url_download): 
  if os.path.exists(file_path) :
    lista_info_leitura.append('Arquivo "{file_path}" já existe!'.format(file_path=file_path))
    return 1
  else :
    lista_info_leitura.append('Baixando o arquivo "{file_path}"...'.format(file_path=file_path))    
    try:
      wget.download(url_download, file_path)
      lista_info_leitura.append('Arquivo "{file_path}" carregado!'.format(file_path=file_path))
      return 1
    except:
      lista_info_leitura.append('Falha ao baixar o arquivo "{file_path}"!'.format(file_path=file_path))
      return 0
try :
  lista_museus = pd.read_csv('lista-museus.csv')
  print("Arquivo carregado com sucesso: lista-museus.csv")
except :
  print("Erro ao ler o arquivo com a lista museus: lista-museus.csv")

Arquivo carregado com sucesso: lista-museus.csv


O interesse nesse projeto como fonte de pesquisa, é que ele foi customizado para atender às necessidades de inventário e catalogação dos acervos museológicos do Ibram, bem como a difusão dessas coleções na internet, ou seja ele dispõe de uma solução difundida e utilizada até o momento para os seguintes museus:

In [8]:
lista_museus

,nome,museu_base_url,url,colecao,formato,qde_arquivos,token_seguranca,qde_itens
0,Museu Histórico Nacional,mhn,mhn.acervos.museus.gov.br,24,csv,6,5e7b4c00ce,502
1,Museu do Diamante,museudodiamante,museudodiamante.acervos.museus.gov.br,11603,csv,10,2263b5c174,130
2,Museu do Ouro,museudoouro,museudoouro.acervos.museus.gov.br,3583,csv,1,3c6b4b3a03,51
3,Museu Victor Meirelles,museuvictormeirelles,museuvictormeirelles.acervos.museus.gov.br,18004,csv,3,bbd0282933,237
4,Museu Regional Casa dos Ottoni,museuregionalcasadosottoni,museuregionalcasadosottoni.acervos.museus.gov.br,31492,csv,5,8eacddfddc,462
5,Museu Arqueológico de Itaipu,museudearqueologiadeitaipu,museudearqueologiadeitaipu.museus.gov.br,94553,csv,11,b8125cf049,1040
6,Museu das Missões,museudasmissoes,museudasmissoes.acervos.museus.gov.br,6409,csv,1,8746d407f4,92
7,Museu da Bandeiras,museusibramgoias,museusibramgoias.acervos.museus.gov.br,865,csv,4,89d3f7d215,361
8,Museu Villa Lobos,museuvillalobos,museuvillalobos.acervos.museus.gov.br,1570,csv,19,b8125cf049,1812
9,useu Casa de Benjamin Constant,museucasabenjaminconstant,museucasabenjaminconstant.acervos.museus.gov.br,5347,csv,11,e4298cb69b,983


Como o objetivo de extrair informações a respeito dos autores de parte desses repositórios elencados (para compará las depois) o intuito de armazenar tais informações, isso porque é necessário realizar uma implementação que irá realizar a recuperação, armazenamento e processamento dos dados obtidos (desenvolvimento de processo automático de vinculação).

In [9]:
#BAIXAR OS ARQUIVOS A PARTIR DA LISTA
from IPython.display import clear_output
arquivos_lidos = []

for link in lista_museus.values :
  qde_arquivos_lidos = 0
  print(link)
  for x in range(1, int(link[5])+1) :
    file_path = 'acervo-{museu}-{paged}.csv'.format(museu=link[1],paged=x)
    url_download = "http://{url}/wp-json/tainacan/v2/collection/{num_collection}/items/?perpage=96&order=DESC&orderby=date&exposer=csv&paged={paged}&_wpnonce={wpnonce}".format(url=link[2],num_collection=link[3],paged=x,wpnonce=link[6])
    print(url_download)
    if baixar_arquivo(file_path,url_download) :
      qde_arquivos_lidos = qde_arquivos_lidos +1
  arquivos_lidos.append(qde_arquivos_lidos)

lista_museus['qde_arquivos_lidos'] = arquivos_lidos
clear_output()
lista_museus

,nome,museu_base_url,url,colecao,formato,qde_arquivos,token_seguranca,qde_itens,qde_arquivos_lidos
0,Museu Histórico Nacional,mhn,mhn.acervos.museus.gov.br,24,csv,6,5e7b4c00ce,502,6
1,Museu do Diamante,museudodiamante,museudodiamante.acervos.museus.gov.br,11603,csv,10,2263b5c174,130,10
2,Museu do Ouro,museudoouro,museudoouro.acervos.museus.gov.br,3583,csv,1,3c6b4b3a03,51,1
3,Museu Victor Meirelles,museuvictormeirelles,museuvictormeirelles.acervos.museus.gov.br,18004,csv,3,bbd0282933,237,3
4,Museu Regional Casa dos Ottoni,museuregionalcasadosottoni,museuregionalcasadosottoni.acervos.museus.gov.br,31492,csv,5,8eacddfddc,462,5
5,Museu Arqueológico de Itaipu,museudearqueologiadeitaipu,museudearqueologiadeitaipu.museus.gov.br,94553,csv,11,b8125cf049,1040,11
6,Museu das Missões,museudasmissoes,museudasmissoes.acervos.museus.gov.br,6409,csv,1,8746d407f4,92,1
7,Museu da Bandeiras,museusibramgoias,museusibramgoias.acervos.museus.gov.br,865,csv,4,89d3f7d215,361,4
8,Museu Villa Lobos,museuvillalobos,museuvillalobos.acervos.museus.gov.br,1570,csv,19,b8125cf049,1812,19
9,useu Casa de Benjamin Constant,museucasabenjaminconstant,museucasabenjaminconstant.acervos.museus.gov.br,5347,csv,11,e4298cb69b,983,11


In [10]:
#PROCESSO DE UNIFICAÇÃO DOS ARQUIVOS
import requests
import re

arquivos_unificados = []
dfs = []
arquivo_unificado = ''

for link in lista_museus.values :
  dfs = []
  for x in range(1, int(link[5])+1) :
    file_path = 'acervo-{museu}-{paged}.csv'.format(museu=link[1],paged=x)
    acervo = pd.read_csv(file_path,sep=';',encoding ='utf-8-sig')
    dfs.append(acervo)
  acervo_concatenado = pd.concat(dfs, ignore_index=True)
  arquivo_unificado = 'acervo_{museu}_completo.csv'.format(museu=link[1])
  try :
    acervo_concatenado.to_csv(arquivo_unificado, index = None, header=True,sep=';',encoding ='utf-8')
    arquivos_unificados.append(arquivo_unificado)
  except :
    arquivos_unificados.append("Erro ao gravar: "+arquivo_unificado)
   

lista_museus["arquivo_unificado"] = arquivos_unificados
lista_museus

,nome,museu_base_url,url,colecao,formato,qde_arquivos,token_seguranca,qde_itens,qde_arquivos_lidos,arquivo_unificado
0,Museu Histórico Nacional,mhn,mhn.acervos.museus.gov.br,24,csv,6,5e7b4c00ce,502,6,acervo_mhn_completo.csv
1,Museu do Diamante,museudodiamante,museudodiamante.acervos.museus.gov.br,11603,csv,10,2263b5c174,130,10,acervo_museudodiamante_completo.csv
2,Museu do Ouro,museudoouro,museudoouro.acervos.museus.gov.br,3583,csv,1,3c6b4b3a03,51,1,acervo_museudoouro_completo.csv
3,Museu Victor Meirelles,museuvictormeirelles,museuvictormeirelles.acervos.museus.gov.br,18004,csv,3,bbd0282933,237,3,acervo_museuvictormeirelles_completo.csv
4,Museu Regional Casa dos Ottoni,museuregionalcasadosottoni,museuregionalcasadosottoni.acervos.museus.gov.br,31492,csv,5,8eacddfddc,462,5,acervo_museuregionalcasadosottoni_completo.csv
5,Museu Arqueológico de Itaipu,museudearqueologiadeitaipu,museudearqueologiadeitaipu.museus.gov.br,94553,csv,11,b8125cf049,1040,11,acervo_museudearqueologiadeitaipu_completo.csv
6,Museu das Missões,museudasmissoes,museudasmissoes.acervos.museus.gov.br,6409,csv,1,8746d407f4,92,1,acervo_museudasmissoes_completo.csv
7,Museu da Bandeiras,museusibramgoias,museusibramgoias.acervos.museus.gov.br,865,csv,4,89d3f7d215,361,4,acervo_museusibramgoias_completo.csv
8,Museu Villa Lobos,museuvillalobos,museuvillalobos.acervos.museus.gov.br,1570,csv,19,b8125cf049,1812,19,acervo_museuvillalobos_completo.csv
9,useu Casa de Benjamin Constant,museucasabenjaminconstant,museucasabenjaminconstant.acervos.museus.gov.br,5347,csv,11,e4298cb69b,983,11,acervo_museucasabenjaminconstant_completo.csv


Além disso, a biblioteca pandas trabalha em uma dinâmica interessante com arquivos no formato CSV, formato esse disponíveis nos repositórios que implementam o Tainacan, esse portanto esse formato de extração e armazenamento, constando que o formato CSV fica bem delimitado ao desenho das 5 estrelas do site https://5stardata.info/.


In [11]:
#VERIFICACAO DAS COLUNAS NECESSÁRIAS
qde_itens_encontrado = []
qde_colunas_encontrado = []
for link in lista_museus.values :
  museu_base_url	= museu=link[1]
  file_path = 'acervo_{museu}_completo.csv'.format(museu=museu_base_url)
  museu_escolha = pd.read_csv(file_path,sep=';',encoding ='utf-8')
  qde_itens_encontrado.append(museu_escolha.shape[0])
  qde_colunas_encontrado.append(museu_escolha.shape[1])
  print("----------------------------------------------------------------------------------------------")
  print("Museu: "+museu_base_url)
  print("Colunas: "+museu_escolha.columns)
  busca_campos = museu_escolha.columns.str.contains("autor|de registro", flags=re.IGNORECASE, regex=True)
  print("Pesquisa de campos com (autor|de registro) retornando (True|False):"+str(busca_campos))
  print("Total para a pesquisa de campos (autor|de registro): "+str(sum(1 for item in busca_campos if item==(True))))

lista_museus["qde_itens_encontrado"] = qde_itens_encontrado
lista_museus["qde_colunas_encontrado"] = qde_colunas_encontrado

----------------------------------------------------------------------------------------------
Museu: mhn
Index(['Colunas: Denominação', 'Colunas: Descrição',
       'Colunas: Número de registro', 'Colunas: Título', 'Colunas: Autor',
       'Colunas: Técnica', 'Colunas: Material', 'Colunas: Forma de aquisição',
       'Colunas: Fonte de aquisição', 'Colunas: Referência de aquisição',
       'Colunas: Local de produção', 'Colunas: Data de Produção',
       'Colunas: Classe', 'Colunas: Estado de conservação',
       'Colunas: Altura (cm)', 'Colunas: Largura (cm)',
       'Colunas: Termos de Indexação', 'Colunas: Exposição',
       'Colunas: Fonte revisada', 'Colunas: País',
       'Colunas: Condições de reprodução'],
      dtype='object')
Pesquisa de campos com (autor|de registro) retornando (True|False):[False False  True False  True False False False False False False False
 False False False False False False False False False]
Total para a pesquisa de campos (autor|de registro): 2
--

In [12]:
lista_museus

,nome,museu_base_url,url,colecao,formato,qde_arquivos,token_seguranca,qde_itens,qde_arquivos_lidos,arquivo_unificado,qde_itens_encontrado,qde_colunas_encontrado
0,Museu Histórico Nacional,mhn,mhn.acervos.museus.gov.br,24,csv,6,5e7b4c00ce,502,6,acervo_mhn_completo.csv,502,21
1,Museu do Diamante,museudodiamante,museudodiamante.acervos.museus.gov.br,11603,csv,10,2263b5c174,130,10,acervo_museudodiamante_completo.csv,895,42
2,Museu do Ouro,museudoouro,museudoouro.acervos.museus.gov.br,3583,csv,1,3c6b4b3a03,51,1,acervo_museudoouro_completo.csv,51,32
3,Museu Victor Meirelles,museuvictormeirelles,museuvictormeirelles.acervos.museus.gov.br,18004,csv,3,bbd0282933,237,3,acervo_museuvictormeirelles_completo.csv,237,34
4,Museu Regional Casa dos Ottoni,museuregionalcasadosottoni,museuregionalcasadosottoni.acervos.museus.gov.br,31492,csv,5,8eacddfddc,462,5,acervo_museuregionalcasadosottoni_completo.csv,462,40
5,Museu Arqueológico de Itaipu,museudearqueologiadeitaipu,museudearqueologiadeitaipu.museus.gov.br,94553,csv,11,b8125cf049,1040,11,acervo_museudearqueologiadeitaipu_completo.csv,1040,36
6,Museu das Missões,museudasmissoes,museudasmissoes.acervos.museus.gov.br,6409,csv,1,8746d407f4,92,1,acervo_museudasmissoes_completo.csv,91,27
7,Museu da Bandeiras,museusibramgoias,museusibramgoias.acervos.museus.gov.br,865,csv,4,89d3f7d215,361,4,acervo_museusibramgoias_completo.csv,361,29
8,Museu Villa Lobos,museuvillalobos,museuvillalobos.acervos.museus.gov.br,1570,csv,19,b8125cf049,1812,19,acervo_museuvillalobos_completo.csv,1812,15
9,useu Casa de Benjamin Constant,museucasabenjaminconstant,museucasabenjaminconstant.acervos.museus.gov.br,5347,csv,11,e4298cb69b,983,11,acervo_museucasabenjaminconstant_completo.csv,983,26


Sabendo que o Wikidata: uma base de conhecimento livre e aberta que pode ser lida e editada por humanos e máquinas. 
https://www.wikidata.org/wiki/Wikidata:Main_Page
e que nele é possível realizar consultas em formato SPARQL, o Wikidata foi a fonte selecionada para a pesquisa e coleta das URIs


In [50]:
#IDENTIFICANDO OS AUTRORES CADASTRADOS PARA CADA MUSEU
import pandas as pd
museu_autores = pd.DataFrame(columns=['rank_autor','museu_base_url','numero_de_registro','autor']) 


def pos_index(t_coluna,t_busca) :
  x = -1
  for pos in t_coluna.str.contains(t_busca, flags=re.IGNORECASE, regex=True) :
      x = x + 1
      if pos :
        return x

for link in lista_museus.values :
  museu_base_url	= museu=link[1]
  file_path = 'acervo_{museu}_completo.csv'.format(museu=museu_base_url)
  museu_escolha = pd.read_csv(file_path,sep=';',encoding ='utf-8')
  museu_escolha["id_acervo"] = museu_escolha.index
  museu_escolha.to_csv('acervo_{museu}_completo_com_id.csv'.format(museu=museu_base_url), index = None, header=True,sep=';',encoding ='utf-8')
  coluna_registro = pos_index(museu_escolha.columns,"de registro")
  coluna_id = pos_index(museu_escolha.columns,"id_acervo")
  coluna_autor = pos_index(museu_escolha.columns,"autor")
    
  for linhas in museu_escolha.values :
    for autor in str(linhas[coluna_autor]).split('|') :
      museu_autores = museu_autores.append({'id_item': linhas[coluna_id],'museu_base_url':museu_base_url, 'autor':autor ,'numero_de_registro': linhas[coluna_registro]}, ignore_index=True)  

museu_autores

,id_item,museu_base_url,numero_de_registro,autor
0,0,mhn,310000,Alcides Naval (Alcides Santos Coelho)
1,1,mhn,305708,Não identificado
2,2,mhn,305707,Não identificado
3,3,mhn,305360,nan
4,4,mhn,305359,Müller (ilegível)
...,...,...,...,...
11107,4617,museudainconfidencia,5,Não identificado
11108,4618,museudainconfidencia,4,Não identificado
11109,4619,museudainconfidencia,3,Não identificado
11110,4620,museudainconfidencia,2,Não identificado


In [0]:
museu_autores["autor_limpo"] = museu_autores["autor"].str.extract('(\w*\s\w*)', expand=True)
museu_autores.to_csv('autores_completo.csv', index = None, header=True,sep=';',encoding ='utf-8')


In [102]:
museu_autores['rank_autor'] = museu_autores['autor'].rank()
autores_contagem = museu_autores.groupby(['autor','rank_autor','museu_base_url']).size().sort_values(axis=0,ascending=False).to_frame(name = 'contagem').reset_index()
autores_contagem

,autor,rank_autor,museu_base_url,contagem
0,Não identificado,5740.0,museudainconfidencia,3530
1,Não identificado,5740.0,museuvillalobos,1620
2,nan,10281.0,museudearqueologiadeitaipu,1040
3,Não identificado,5740.0,museucasabenjaminconstant,925
4,Não identificada,2102.5,museudodiamante,664
...,...,...,...,...
602,Jorge de Lima,1487.0,mhn,1
603,José Alberto Nemer,1488.0,museudainconfidencia,1
604,José Aparicio y Inglada (atribuído),1489.0,mhn,1
605,José Barreto de Menezes,1490.0,mhn,1


Compreendendo que as informações disponíveis no wikidata são em grafo, ou seja as informações são recuperadas no conceito de sujeito, predicado objeto e a consultas serão realizadas n consultas em SPARQL, uma para cada autor identificado foi delimitado as seguintes propriedades de acerca de cada instância da seguinte forma:
 


*   instância de (P31) ser humano (Q5)
*   data de nascimento (P569)
*   data de morte (P570)
*   género (P21)
*   ocupação (P106)

 
Em resumo o conjunto de consultas, irá recuperar as instâncias de seres humanos que correspondentes aos nomes dos autores, com as informações da data de nascimento, data de morte (falecimento), sexo ou gênero e ocupação e área de trabalho e armazená-las localmente para posterior tratamento com a biblioteca Pandas. 


In [0]:
from qwikidata.sparql import (get_subclasses_of_item,
                              return_sparql_query_results)
import time
from tqdm import tqdm

def consulta_sparql_wikidata(autor) :
  time.sleep(4)
  sparql_query = (
    "SELECT ?item ?itemLabel ?birthdate ?deathdate ?genderLabel ?ocupation \n"
    "WHERE \n"
    "  { "
    "?item wdt:P31 wd:Q5. \n"
    "?item ?label \"%s\"@pt-br. \n"
    "OPTIONAL { ?item wdt:P569 ?birthdate }. \n"
    "OPTIONAL { ?item wdt:P570 ?deathdate }. \n"
    "OPTIONAL { ?item wdt:P21 ?genderLabel }. \n"
    "OPTIONAL { ?item wdt:P106 ?ocupation } \n"
    "SERVICE wikibase:label { bd:serviceParam wikibase:language \"pt-br\". } \n"
    " } \n"
    "LIMIT 10"
    % (autor))
  try : 
    res = return_sparql_query_results(sparql_query)
  except ValueError :
    return 'Erro'
  return res
 
autores_wikidata = pd.DataFrame(columns=['museu_base_url','rank_autor','nm_autor', 'uri', 'dt_nascimento','dt_morte','ocupacao','qde_encontrado','json','ic_erro'])
pbar = tqdm(total=autores_contagem['autor'].count())

for autor in autores_contagem.values :
  
  nm_autor = str(autor[0])
  nm_museu = str(autor[2])
  rank_autor = str(autor[1])
  data_json = consulta_sparql_wikidata(nm_autor)
  dt_nascimento = '' 
  dt_morte = ''
  ocupacao = ''
  uri = ''
  qde_encontrado = 0
  dados_json = str(data_json)
  
  lista_exececoes = []
  lista_erros = []

  if isinstance(data_json,dict) :
    qde_encontrado = len(data_json['results']['bindings'])

    if qde_encontrado > 0 :
      try:
          uri = data_json['results']['bindings'][0]['item']['value']
      except KeyError as ke:
        lista_exececoes.append("Warning: uri não encontrado para {nm_autor}".format(nm_autor=nm_autor))
      try:
         dt_nascimento = data_json['results']['bindings'][0]['birthdate']['value']
      except KeyError as ke:
        lista_exececoes.append("Warning: data de nascimento não encontrado para {nm_autor}".format(nm_autor=nm_autor))
      try:
         dt_morte = data_json['results']['bindings'][0]['deathdate']['value'] 
      except KeyError as ke:
        lista_exececoes.append("Warning: data de morte não encontrado para {nm_autor}".format(nm_autor=nm_autor))
      try:
         ocupacao = data_json['results']['bindings'][0]['ocupation']['value'] 
      except KeyError as ke:
        lista_exececoes.append("Warning: ocupação não encontrado para {nm_autor}".format(nm_autor=nm_autor))
    ic_erro = 0
  else :
    ic_erro = 1
    lista_erros.append("Erro: falha ao recuperar no wikidata {nm_autor}".format(nm_autor=nm_autor))  
  
  autores_wikidata = autores_wikidata.append({'museu_base_url': nm_museu ,'rank_autor':rank_autor,'nm_autor': nm_autor, 'uri': uri, 'dt_nascimento':dt_nascimento,'dt_morte':dt_morte,'ocupacao':ocupacao,'qde_encontrado':qde_encontrado,'json':'str(dados_json)','ic_erro':ic_erro}, ignore_index=True)
  clear_output()
  pbar.update(1)

100%|██████████| 607/607 [50:20<00:00,  4.95s/it]

In [0]:
autores_wikidata.to_csv('autores_wikidata.csv', index = None, header=True,sep=';',encoding ='utf-8')

Mesmo sendo um conjunto restrito de informações os atributos eleitos, conseguem dar uma perspectiva de algumas verificações:
 
*   Quantas URIs, foram encontradas ou não para a pesquisa do nome do autor? 
*   Dos resultados encontrados, quantos possuem as propriedades de nascimento,	morte, gênero, ocupação e área de trabalho?
*   Qual a lista URIs de propriedades identificadas para gênero, ocupação?












In [180]:
#URIs que foram encontradas ou não para a pesquisa do nome do autor?
autores_wikidata_encontrados = autores_wikidata[['museu_base_url','uri']].groupby('museu_base_url').count().reset_index().set_index('museu_base_url').merge(autores_wikidata[['museu_base_url','uri']].where(autores_wikidata['uri'] != '').groupby('museu_base_url').count().reset_index().set_index('museu_base_url'),on='museu_base_url',how='left').reset_index()
autores_wikidata_encontrados['uri_z'] = autores_wikidata_encontrados['uri_x']-autores_wikidata_encontrados['uri_y']
autores_wikidata_encontrados.columns=['museu_base_url','total_autores','autores_encontrados','autores_nao_encontrados']
autores_wikidata_encontrados

,museu_base_url,total_autores,autores_encontrados,autores_nao_encontrados
0,mhn,214,133.0,81.0
1,museucasabenjaminconstant,37,6.0,31.0
2,museudainconfidencia,135,10.0,125.0
3,museudasmissoes,3,NaN,NaN
4,museudearqueologiadeitaipu,1,NaN,NaN
5,museudodiamante,77,NaN,NaN
6,museudoouro,3,NaN,NaN
7,museuregionalcasadosottoni,43,NaN,NaN
8,museusibramgoias,23,NaN,NaN
9,museuvictormeirelles,68,59.0,9.0


In [188]:
autores_wikidata[['museu_base_url','dt_nascimento']].where(autores_wikidata['dt_nascimento']!='NaN')

,museu_base_url,dt_nascimento
0,museudainconfidencia,
1,museuvillalobos,
2,museudearqueologiadeitaipu,
3,museucasabenjaminconstant,
4,museudodiamante,
...,...,...
602,mhn,1893-04-23T00:00:00Z
603,museudainconfidencia,
604,mhn,
605,mhn,
